<a href="https://colab.research.google.com/github/douggsz/dataset-treinamento/blob/main/transfer_learn_cat_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os # Importa o módulo os para interagir com o sistema operacional.
import zipfile # Importa o módulo zipfile para trabalhar com arquivos zip.
import random # Importa o módulo random para gerar números aleatórios.
import tensorflow as tf # Importa o módulo TensorFlow para aprendizado de máquina.

from tensorflow.keras.optimizers import RMSprop # Importa o otimizador RMSprop do Keras.
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Importa o ImageDataGenerator para pré-processamento de imagens.
from shutil import copyfile # Importa a função copyfile do módulo shutil para copiar arquivos.

In [ ]:
!wget --no-check-certificate \
    "https://data.caltech.edu/records/mzrjq-6wc02/files/caltech-101.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

--2025-01-20 14:04:12--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.197.2.18, 2600:1408:c400:c84::317f, 2600:1408:c400:c81::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.197.2.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.67M   233MB/s    in 3.4s    

2025-01-20 14:04:15 (229 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824887076/824887076]



In [ ]:
print(len(os.listdir('/tmp/PetImages/Cat/')))
print(len(os.listdir('/tmp/PetImages/Dog/')))

NotADirectoryError: [Errno 20] Not a directory: '/tmp/101'

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)


CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)


666.jpg is zero length, so ignoring.
11702.jpg is zero length, so ignoring.


In [ ]:
print(len(os.listdir('/tmp/cats-v-dogs/training/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))

12362
12364
2362
2364


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['acc'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
TRAINING_DIR = "/tmp/cats-v-dogs/training/"
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=250,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

VALIDATION_DIR = "/tmp/cats-v-dogs/testing/"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=250,
                                                              class_mode='binary',
                                                              target_size=(150, 150))

Found 24724 images belonging to 2 classes.
Found 4725 images belonging to 2 classes.


In [ ]:
history = model.fit(train_generator, epochs=15, steps_per_epoch=90,
                    validation_data=validation_generator, validation_steps=6)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


21/90 ━━━━━━━━━━━━━━━━━━━━ 7:11 6s/step - acc: 0.4964 - loss: 1.7073

/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


90/90 ━━━━━━━━━━━━━━━━━━━━ 574s 6s/step - acc: 0.5333 - loss: 1.0498 - val_acc: 0.6553 - val_loss: 0.6406
Epoch 2/15
 9/90 ━━━━━━━━━━━━━━━━━━━━ 7:40 6s/step - acc: 0.6373 - loss: 0.6479

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


90/90 ━━━━━━━━━━━━━━━━━━━━ 64s 659ms/step - acc: 0.6285 - loss: 0.6484 - val_acc: 0.6120 - val_loss: 0.6457
Epoch 3/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 561s 6s/step - acc: 0.6671 - loss: 0.6069 - val_acc: 0.7240 - val_loss: 0.5458
Epoch 4/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 55s 531ms/step - acc: 0.7029 - loss: 0.5632 - val_acc: 0.7689 - val_loss: 0.5329
Epoch 5/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 566s 6s/step - acc: 0.7109 - loss: 0.5559 - val_acc: 0.7647 - val_loss: 0.4985
Epoch 6/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 64s 644ms/step - acc: 0.7143 - loss: 0.5529 - val_acc: 0.7607 - val_loss: 0.5117
Epoch 7/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 616s 6s/step - acc: 0.7483 - loss: 0.5073 - val_acc: 0.7720 - val_loss: 0.4817
Epoch 8/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 53s 532ms/step - acc: 0.7564 - loss: 0.5003 - val_acc: 0.7733 - val_loss: 0.4511
Epoch 9/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 569s 6s/step - acc: 0.7740 - loss: 0.4695 - val_acc: 0.8013 - val_loss: 0.4363
Epoch 10/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 64s 634ms/step - acc: 0.7839